<a href="https://colab.research.google.com/github/Carapet07/Data-Science-projects/blob/main/First_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##My first NLP project

In [1]:
import os, pathlib, shutil, random
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import TextVectorization

In [2]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz
!rm -r aclImdb/train/unsup
!cat aclImdb/train/pos/4077_10.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  3170k      0  0:00:25  0:00:25 --:--:-- 5198k
I first saw this back in the early 90s on UK TV, i did like it then but i missed the chance to tape it, many years passed but the film always stuck with me and i lost hope of seeing it TV again, the main thing that stuck with me was the end, the hole castle part really touched me, its easy to watch, has a great story, great music, the list goes on and on, its OK me saying how good it is but everyone will take there own best bits away with them once they have seen it, yes the animation is top notch and beautiful to watch, it does show its age in a very few parts but that has now become part of it beauty, i am so glad it has came out on DVD as it is one of my top 10 films of all time. Buy it or rent it just see it, best viewing is at night alone with drin

In [3]:
base_dir = pathlib.Path("aclImdb")
val_dir = base_dir / "val"
train_dir = base_dir / "train"

# Create a new directory for validation that will store 20% of the training dir
for category in ("neg", "pos"):
  os.makedirs(val_dir / category)
  files = os.listdir(train_dir / category)
  random.Random(1337).shuffle(files)
  num_val_samples = int(0.2 * len(files))
  val_files = files[-num_val_samples:]

  for fname in val_files:
    shutil.move(train_dir / category / fname,
                val_dir / category / fname)

In [4]:
batch_size = 32

# create batches that will yield text data belonging to 2 classes
train_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/train", batch_size=batch_size
)
test_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/val", batch_size=batch_size
)
val_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/val", batch_size=batch_size
)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.


In [5]:
for text_batch, label_batch in train_ds.take(1):  # Take one batch
    for i in range(5):  # Show the first 5 examples
        print(f"Text: {text_batch.numpy()[i].decode('utf-8')}")
        print(f"Label: {label_batch.numpy()[i]}")
        print("=" * 50)


Text: "Look, I know this may suck right now, but pain is temporary, film is forever. Whatever you do right now is burned into celluloid for all time and for thousands of years to come."  Robert De Niro<br /><br />This was initially a film for Steven Spielberg, the director hiring several screenwriters to adjust the screenplay so that it more suited his themes. And so we have a dysfunctional family that is threatened by a deranged monster in the form of a recently released from prison Robert De Niro. Like "Jurassic Park", "Poltergeist" and "War of the Worlds", the dysfunctional family bands together to defeat the beast, the beast being the creation of the father, a lawyer who failed to defend De Niro during his trial. <br /><br />In typical Spielberg fashion, the father kills the monster that undermines the family and is then promptly absolved of his sins. Like Oscar Schindler, Robin Williams, Sam Niel, Tom Cruise and virtually every "daddy" figure in Spielberg land, the father reclaim

In [6]:
max_length = 600 # max length of a movie review
max_tokens = 20000 # total number of reviews

# Create a TextVectorization layer that maps text features to integer sequences
# eg: "dog": [0.2, 0.5, -0.7, 0.21... embedding_dim]
text_vectorization = TextVectorization(
    max_tokens=max_tokens,
    output_mode='int',
    output_sequence_length=max_length
)

# The train_ds is a dic that contains words and their numbers
# text_only_train_ds stores only the words
text_only_train_ds = train_ds.map(lambda x, y: x)

# This line adapts the text_only_train_ds vector by assinging each word it's unique embedding layer
text_vectorization.adapt(text_only_train_ds)

# apply it to every dataset we have
int_train_ds = train_ds.map(lambda x, y: (text_vectorization(x), y),
                            num_parallel_calls=4)
int_test_ds = test_ds.map(lambda x, y: (text_vectorization(x), y),
                            num_parallel_calls=4)
int_val_ds = val_ds.map(lambda x, y: (text_vectorization(x), y),
                            num_parallel_calls=4)

In [7]:
for inputs, targets in train_ds:
  print("input shape: ", inputs.shape)
  print("input dtype: ", inputs.dtype)
  print("targets shape: ", targets.shape)
  print("targets dtype: ", targets.dtype)
  print('inputs[0]: ', inputs[0])
  print('targets[0]: ', targets[0])
  break

input shape:  (32,)
input dtype:  <dtype: 'string'>
targets shape:  (32,)
targets dtype:  <dtype: 'int32'>
inputs[0]:  tf.Tensor(b"Before I start, I _love_ Eddie Izzard. I think he's one of the funniest stand-ups around today. Possibly that means I'm going into this with too high expectations, but I just didn't find Eddie funny in this outing.<br /><br />I think the main problem is Eddie is trying too hard to be Eddie. Everyone knows him as a completely irrelevant comic, and we all love him for it. But in Circle, he appears to be going more for irrelevant than funny, and completely lost me in places. Many of the topics he covers he has covered before - I even think I recognised a few recycled jokes in there.<br /><br />If you buy the DVD you'll find a behind-the-scenes look at Eddie's tour (interesting in places, but not very funny), and a French language version of one of his shows. Die-hards will enjoy seeing Eddie in a different language, but subtitled comedy isn't very funny.<br />

Parse the unzipped file (.txt file) to build an index that maps words(as strings) to their words reprezentation

In [11]:
glove_100d_path = "glove.6B.100d.txt"
# 1: Create an embedding dictionary
embeddings_index = {}

# 2: Parse the GloVe txt file. Add each word and it's embedding into emeddings_index dictionary
with open(glove_100d_path) as f:
  for line in f:
    word, coefs = line.split(maxsplit=1)
    coefs = np.fromstring(coefs, "f", sep=" ") # converts coefs strings into float32
    embeddings_index[word] = coefs

print(f'found {len(embeddings_index)} words')

found 400000 word 


Next lets build an embedding matrix that will be loaded into an Embedding layer

In [ ]:
embedding_dim = 100

# 1: Get the vocabulary from the TextVectorizaton layer
vocabulary = text_vectorization.get_vocabulary()

# 2: Assign to each word a uniquie number
word_index = dict(zip(vocabulary, range(len(vocabulary))))

# 3: Create a matrix filled with zeros
embedding_matrix = np.zeros((max_tokens, embedding_dim))

# 4: For each word in the vocabulary find it's GloVe embedding
for word, i in word_index.items():
  if i < max_tokens:
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
      embedding_matrix[word] = embedding_vector


This dataset yields inputs that are tf.string tensors and targets that are int32 tensors encoding the value "0" or "1"


All set, now let's try learnign something from this data


In [8]:
inputs = keras.Input(shape=(None, ), dtype='int64')
embedded = layers.Embedding(input_dim=max_tokens, output_dim=256, mask_zero=True)(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model = keras.Model(inputs, outputs)

model.compile(
    optimizer='rmsprop',
    loss='binary_crossentropy',
    metrics=['accuracy']
)
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, None)           │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding (Embedding)     │ (None, None, 256)      │      5,120,000 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ not_equal (NotEqual)      │ (None, None)           │              0 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bidirectional             │ (None, 64)             │         73,984 │ embedding[0][0],       │
│ (Bidirectional)           │                        │                │ not_equal[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout (Dropout)         │ (None, 64)             │              0 │ bidirectional[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 1)              │             65 │ dropout[0][0]          │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 5,194,049 (19.81 MB)

 Trainable params: 5,194,049 (19.81 MB)

 Non-trainable params: 0 (0.00 B)

In [9]:
callbacks = [
    keras.callbacks.ModelCheckpoint("embeddings_bidir_gru.keras",
                                      save_best_only=True)
    ]

history = model.fit(int_train_ds.cache(),
  validation_data=int_val_ds.cache(),
  epochs=10,
  callbacks=callbacks)

print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 39s 55ms/step - accuracy: 0.6881 - loss: 0.5646 - val_accuracy: 0.8710 - val_loss: 0.3179
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 28s 44ms/step - accuracy: 0.8744 - loss: 0.3153 - val_accuracy: 0.8754 - val_loss: 0.3030
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 28s 44ms/step - accuracy: 0.9043 - loss: 0.2493 - val_accuracy: 0.8684 - val_loss: 0.3530
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 27s 43ms/step - accuracy: 0.9279 - loss: 0.1957 - val_accuracy: 0.8796 - val_loss: 0.3454
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 27s 43ms/step - accuracy: 0.9425 - loss: 0.1604 - val_accuracy: 0.8340 - val_loss: 0.5525
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 26s 42ms/step - accuracy: 0.9563 - loss: 0.1216 - val_accuracy: 0.8762 - val_loss: 0.4038
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 26s 42ms/step - accuracy: 0.9705 - loss: 0.0885 - val_accuracy: 0.8732 - val_loss: 0.4175
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 29s 46ms/step - accuracy: 0.9787 - loss: 0.0654 - 

In [10]:
# downlaod GloVe precomputed database for words ebedding task
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip

--2025-02-14 19:17:45--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2025-02-14 19:17:46--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2025-02-14 19:17:46--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov